In [3]:
import os
import warnings
warnings.filterwarnings("ignore")

### base directory

In [4]:
base_dir = 'people'

### Making Folder for each train and test

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_images_by_gender(input_folder, output_folder, train_ratio=0.8):
    # Define gender categories
    categories = ['MEN', 'WOMAN']

    for category in categories:
        category_path = os.path.join(input_folder, category)
        if not os.path.exists(category_path):
            print(f"Directory {category_path} does not exist.")
            continue

        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        # Split images into train and test sets
        train_images, test_images = train_test_split(images, train_size=train_ratio, random_state=42)
        
        # Define train and test paths
        train_path = os.path.join(output_folder, 'train', category)
        test_path = os.path.join(output_folder, 'test', category)
        
        # Create directories if they don't exist
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)
        
        # Copy images to train folder
        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_path, img))
        
        # Copy images to test folder
        for img in test_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_path, img))

# Define paths
input_folder = 'dataset'
output_folder = 'people'

# Split images
split_images_by_gender(input_folder, output_folder, train_ratio=0.8)


In [6]:
men_dir = os.path.join(base_dir, 'train')
women_dir = os.path.join(base_dir, 'test')

### import modules

In [7]:
import tensorflow
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, layers, models


### train data with data augmentation

In [8]:
train_datagen = ImageDataGenerator(
    
    rescale = 1./255,
    rotation_range = 40,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    
    men_dir,
    target_size = (28,28),
    batch_size = 20,
    class_mode = 'binary'
    
)

valid_generator = test_datagen.flow_from_directory(
    
    women_dir,
    target_size = (28,28),
    batch_size = 20,
    class_mode = 'binary'
)

Found 1031 images belonging to 2 classes.


Found 260 images belonging to 2 classes.


### Apply model 

In [10]:
model = models.Sequential([
    Input(shape=(28, 28, 3)),
    layers.Conv2D(32,(3, 3), activation= 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32,(3, 3), activation= 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
     layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid') #output layer 
])

### summary of complete data

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       410,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,049 (1.85 MB)

 Trainable params: 486,049 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

### complie the model

In [12]:
model.compile(
    
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

### train the model

In [13]:
model.fit(
     
    train_generator,
    epochs=10,
    validation_data=valid_generator
)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 22s 334ms/step - accuracy: 0.5392 - loss: 0.7022 - val_accuracy: 0.5115 - val_loss: 0.6881
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 354ms/step - accuracy: 0.5360 - loss: 0.6862 - val_accuracy: 0.6154 - val_loss: 0.6500
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 347ms/step - accuracy: 0.6754 - loss: 0.6070 - val_accuracy: 0.6115 - val_loss: 0.6441
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 347ms/step - accuracy: 0.6611 - loss: 0.6145 - val_accuracy: 0.6423 - val_loss: 0.6592
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 24s 438ms/step - accuracy: 0.6943 - loss: 0.5966 - val_accuracy: 0.6538 - val_loss: 0.6143
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 29s 511ms/step - accuracy: 0.6754 - loss: 0.5836 - val_accuracy: 0.6846 - val_loss: 0.6086
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 26s 442ms/step - accuracy: 0.6886 - loss: 0.5661 - val_accuracy: 0.7038 - val_loss: 0.5538
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 332ms/step - accuracy: 0.7079 - loss: 0.5456 - val_accu

In [42]:
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - accuracy: 0.7369 - loss: 0.5153
Test accuracy: 0.7153846025466919


### Almost Accuracy using CNN is 71%

### Now Fature Extraction transfer learning

In [14]:
import tensorflow
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16

### freeze CNN layers

In [15]:
conv_base = VGG16(

    include_top = False,
    input_shape = (150,150,3)
)

### Set own size od dense layers

In [16]:
model = Sequential()
model.add(conv_base),
model.add(Flatten()),
model.add(Dense(512, activation= 'relu')),
model.add(Dense(1, activation= 'sigmoid'))

In [17]:
conv_base.trainable = False

In [18]:
from tensorflow.keras.preprocessing.image import  ImageDataGenerator

### Train data with augmentation

In [19]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'people/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'people/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 1031 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


#### compile the model

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#### Start training of model

In [21]:
history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 388s 11s/step - accuracy: 0.6017 - loss: 1.5663 - val_accuracy: 0.8154 - val_loss: 0.4365
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 325s 10s/step - accuracy: 0.8820 - loss: 0.2640 - val_accuracy: 0.8500 - val_loss: 0.3082
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 357s 11s/step - accuracy: 0.9262 - loss: 0.1866 - val_accuracy: 0.8654 - val_loss: 0.3504
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 357s 11s/step - accuracy: 0.9463 - loss: 0.1422 - val_accuracy: 0.8692 - val_loss: 0.3108
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 368s 10s/step - accuracy: 0.9627 - loss: 0.1039 - val_accuracy: 0.9000 - val_loss: 0.3265


### Using Feature Extraction Transfer learning Accuracy 87%
### using 5 epochs

In [22]:
model.save('people.h5')